In [2]:
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from utils.inkml2img import convert_dir
from tqdm.auto import tqdm
from data.dataset import Im2LatexDataset
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from model.vit import ViT
import lightning as L
import wandb
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from model.decoder import Decoder, DecoderAttention, DecoderTransformerBlock
from model.model import Img2MathModel


In [ ]:

img_dims = [256, 256]
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", img_dims=img_dims, batch_size=2, device=torch.device('cpu'))
batch = next(iter(data.train))
img, label = batch
vocab_size = len(data.tokenizer.get_vocab())
model = Img2MathModel.load_from_checkpoint('../train/img2math/362edsd8/checkpoints/epoch=9-step=65180.ckpt', map_location=torch.device('cpu'))
model(img)
#model.eval()
#logits, _ = model(img)
#torch.argmax(logits)
#val_imgs = img
#model.training_step(batch, 1)

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/utilities/migration/utils.py:51: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.4, which is newer than your current Lightning version: v2.0.3
  rank_zero_warn(


0 torch.Size([256, 2160]) torch.Size([2, 256])
1 torch.Size([256, 2160]) torch.Size([2, 256])
2 torch.Size([256, 2160]) torch.Size([2, 256])
3 torch.Size([256, 2160]) torch.Size([2, 256])
4 torch.Size([256, 2160]) torch.Size([2, 256])
5 torch.Size([256, 2160]) torch.Size([2, 256])
6 torch.Size([256, 2160]) torch.Size([2, 256])
7 torch.Size([256, 2160]) torch.Size([2, 256])
8 torch.Size([256, 2160]) torch.Size([2, 256])
9 torch.Size([256, 2160]) torch.Size([2, 256])
10 torch.Size([256, 2160]) torch.Size([2, 256])
11 torch.Size([256, 2160]) torch.Size([2, 256])
12 torch.Size([256, 2160]) torch.Size([2, 256])
13 torch.Size([256, 2160]) torch.Size([2, 256])
14 torch.Size([256, 2160]) torch.Size([2, 256])
15 torch.Size([256, 2160]) torch.Size([2, 256])
16 torch.Size([256, 2160]) torch.Size([2, 256])
17 torch.Size([256, 2160]) torch.Size([2, 256])
18 torch.Size([256, 2160]) torch.Size([2, 256])
19 torch.Size([256, 2160]) torch.Size([2, 256])
20 torch.Size([256, 2160]) torch.Size([2, 256])
21

171 torch.Size([256, 2160]) torch.Size([2, 256])
172 torch.Size([256, 2160]) torch.Size([2, 256])
173 torch.Size([256, 2160]) torch.Size([2, 256])
174 torch.Size([256, 2160]) torch.Size([2, 256])
175 torch.Size([256, 2160]) torch.Size([2, 256])
176 torch.Size([256, 2160]) torch.Size([2, 256])
177 torch.Size([256, 2160]) torch.Size([2, 256])
178 torch.Size([256, 2160]) torch.Size([2, 256])
179 torch.Size([256, 2160]) torch.Size([2, 256])
180 torch.Size([256, 2160]) torch.Size([2, 256])
181 torch.Size([256, 2160]) torch.Size([2, 256])
182 torch.Size([256, 2160]) torch.Size([2, 256])
183 torch.Size([256, 2160]) torch.Size([2, 256])
184 torch.Size([256, 2160]) torch.Size([2, 256])
185 torch.Size([256, 2160]) torch.Size([2, 256])
186 torch.Size([256, 2160]) torch.Size([2, 256])
187 torch.Size([256, 2160]) torch.Size([2, 256])
188 torch.Size([256, 2160]) torch.Size([2, 256])
189 torch.Size([256, 2160]) torch.Size([2, 256])
190 torch.Size([256, 2160]) torch.Size([2, 256])
191 torch.Size([256,

In [3]:
def decode(tokenizer, sequence):
    dec = [tokenizer.decode(tok) for tok in sequence]
    return ''.join([detok.replace('Ġ', ' ') for detok in dec])

#preds = [model.generate(img) for img in val_imgs]
tokenizer = PreTrainedTokenizerFast(tokenizer_file='../data/tokenizer.json', padding_side='left')

seq = model.generate(val_imgs[0])
decode(tokenizer, seq)



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 #preds = [model.generate(img) for img in val_imgs]                                          │
│    6 tokenizer = PreTrainedTokenizerFast(tokenizer_file='../data/tokenizer.json', padding_sid    │
│    7                                                                                             │
│ ❱  8 seq = model.generate(val_imgs[0])                                                           │
│    9 decode(tokenizer, seq)                                                                      │
│   10                                                                                             │
│   11                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'val_imgs' is not defined

In [3]:
TOKENIZER_FILE = '../data/tokenizer.json'

def decode(tokenizer, sequence):
    dec = [tokenizer.decode(tok) for tok in sequence]
    return ''.join([detok.replace('Ġ', ' ') for detok in dec])

class ImagePredictionLogger(L.Callback):
    
    def __init__(self, val_samples, num_samples=10):
        super().__init__()
        self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_FILE, padding_side='left')
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = [decode(self.tokenizer, label) for label in self.val_labels['input_ids'][:num_samples]]


    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)

        pred_tokens = [pl_module.generate(x) for x in val_imgs]
        preds = [decode(self.tokenizer, pred) for pred in pred_tokens]

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                         for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
        })

logger = WandbLogger(project='img2math')

trainer = L.Trainer(limit_train_batches=10, max_epochs=1, log_every_n_steps=10, deterministic=True,
                    logger=logger, accelerator='mps', callbacks=[ImagePredictionLogger(batch)])

trainer.fit(model, data.train, data.test)
wandb.finish()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: baolong. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | encoder | ViT     | 8.9 M 
1 | decoder | Decoder | 14.9 M
------------------------------------
23.9 M    Trainable params
0         Non-trainable params
23.9 M    Total params
95.443    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


epoch,▁▁
global_step,▁█
train/loss,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/loss_epoch,▁
val/loss_step,▂▄▃▃▃▂▃▃▆▂▄▂▂▂▃▃█▁▂▂▁▃▄▃▂▆▂▂▁▃▃▂▄▃▃▂▇▃▃▁
epoch,0
global_step,10
train/loss,0.00864
trainer/global_step,9
val/loss_epoch,0.44301


In [17]:
device = torch.device('mps')
a = torch.randn(256, 512).to(device)

for i in range(100):
    b = torch.randn(256, 512).to(device)
    c = a @ b.T
    print(f"mem usage: {torch._C._mps_currentAllocatedMemory()}")


mem usage: 1310720
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1